In [2]:
from abia_energia import*
from __future__ import annotations
from typing import List, Generator, Set
import timeit
from search import Problem, hill_climbing
from math import*
from collections import*


In [3]:
centrales = Centrales([5, 10, 5], 42)

In [4]:
clientes = Clientes(500, [0.2, 0.3, 0.5], 0.5, 42)

In [5]:
class ProblemParameters(object):
    def __init__(self, num_centrals: int, num_clients: int, clients: list, centrals: list):
        self.num_centrals = num_centrals # Màxim número de contenidors
        self.num_clients = num_clients # Màxim número de paquets
        self.centrals = centrals # Llista d'objectes central
        self.clients = clients # Llista d'objectes clients
        
        
        
        self.matriu_c=[[None for j in range(self.num_centrals)] for i in range(self.num_clients)]
                
        for i in range(self.num_clients): 
            for j in range(self.num_centrals):
                x,y=clientes[i].CoordX, clientes[i].CoordY
                xc,yc=centrales[j].CoordX,centrales[j].CoordY
                distancia=dist([x,y],[xc,yc])

                consumo=clientes[i].Consumo
                consumoT= consumo + consumo*VEnergia.loss(distancia)
                self.matriu_c[i][j]= consumoT

        
    def __repr__(self):
        return f"Params(centrals={self.centrals}, clients={self.clients})"
        
class EnergyOperator(object):
    pass

class MoveClient(EnergyOperator):
    def __init__(self, client_i: int, central_j: int, central_k: int):
        self.client_i = client_i
        self.central_j = central_j
        self.central_k = central_k
        
    def __repr__(self) -> str:
        return f"Move client {self.client_i} from central {self.central_j} to central {self.central_k}"
    
class SwapClient(EnergyOperator):
    def __init__(self, client_i: int, client_j: int):
        self.client_i = client_i
        self.client_j = client_j
        
    def __repr__(self) -> str:
        return f"Swap clients {self.client_i} and {self.client_j}"
    
class StateRepresentation(object):
    def __init__(self, params: ProblemParameters, v_a: List[int], produccion_central: List[int] = None):
        self.params = params
        self.v_a = v_a
        if produccion_central is None:
            self.produccion_central=[0 for j in range(self.params.num_centrals)]

            for client_i, central_j in enumerate(self.v_a):
                consum_client_i = self.params.matriu_c[client_i][central_j]
                self.produccion_central[central_j] += consum_client_i
        else:
            self.produccion_central = produccion_central
        
    def copy(self) -> StateRepresentation:
        return StateRepresentation(self.params, self.v_a.copy() , self.produccion_central.copy())
    
    def __repr__(self) -> str:
        return f"v_a={self.v_a} | Produccion_central={self.produccion_central} | {self.params}"
    
    def generate_actions(self) -> Generator[EnergyOperator,None,None]:
        for client_i, central_j in enumerate(self.v_a):     
            consumo_cliente_i= self.params.matriu_c[client_i][central_j]
            for central_k, prod_i in enumerate(self.produccion_central):
                if central_j != central_k and self.params.centrals[central_j].Produccion - prod_i >= consumo_cliente_i:
                    yield MoveClient(client_i, central_j, central_k)
                    
        # Intercanviar paquets
        for client_i in range(self.params.num_clients):
            for client_j in range(self.params.num_clients):
                if client_i != client_j:
                    central_i = self.v_a[client_i]
                    central_j = self.v_a[client_j]
                    
                    consumt_i_i =  self.params.matriu_c[client_i][central_i]
                    consumt_j_j =  self.params.matriu_c[client_j][central_j]
                    
                    # Condició: hi ha espai lliure suficient per fer l'intercanvi
                    # (Espai lliure del contenidor + espai que deixa el paquet >= espai del nou paquet)
                    if self.produccion_central[central_i] + consumt_j_j >= self.produccion_central[central_i] + consumt_i_i and self.produccion_central[central_j] + consumt_i_i >= self.produccion_central[central_j] + consumt_j_j:
                        yield SwapClient(client_i, client_j)
                        
        #Afegir client
             
    def apply_action(self, action: EnergyOperator) -> StateRepresentation:
        new_state = self.copy()
        if isinstance(action, MoveClient):
            client_i =  action.client_i
            central_j = action.central_j
            central_k = action.central_k
            
            new_state.v_a[client_i] = central_k
            
            new_state.produccion_central[central_j] -= self.params.matriu_c[client_i][central_j]
            new_state.produccion_central[central_k] += self.params.matriu_c[client_i][central_k]
            
            

                
        elif isinstance(action, SwapClient):
            client_i = action.client_i
            client_j = action.client_j
            
            central_k = new_state.v_a[client_i]
            central_l = new_state.v_a[client_j]
            
            new_state.v_a[client_i] = central_k
            new_state.v_a[client_j] = central_l
            
            new_state.produccion_central[central_k] -= self.params.matriu_c[client_i][central_k]
            new_state.produccion_central[central_l] -= self.params.matriu_c[client_j][central_l]
            new_state.produccion_central[central_k] += self.params.matriu_c[client_j][central_k]
            new_state.produccion_central[central_l] += self.params.matriu_c[client_i][central_l]
            
        return new_state

    
    def heuristic(self) -> float:
        benefici=0
        for index_client, index_central in enumerate(self.v_a):
            
            client=self.params.clients[index_client]
            central=self.params.centrals[index_central]
            
            consumMonetitzable = client.Consumo
            consumTotalClient = self.params.matriu_c[index_client][index_central]
            
            garant_nogarant = VEnergia.tarifa_cliente_no_garantizada if client.Contrato else VEnergia.tarifa_cliente_garantizada
            costesProd= VEnergia.costs_production_mw(central.Tipo)+VEnergia.daily_cost(central.Tipo) 
            
            #tarifa_cliente_penalizacion
            
            benefici+= consumMonetitzable * garant_nogarant(client.Tipo) - (consumTotalClient*costesProd)
            
        for index_central in set(self.v_a):
            central=self.params.centrals[index_central]
            benefici-= VEnergia.daily_cost(central.Tipo)
        return benefici
        
        
    
class EnergyProblem(Problem):
    def __init__(self, initial_state: StateRepresentation):
        super().__init__(initial_state)
        
    def actions(self, state: StateRepresentation) -> Generator[EnergyOperator,None,None]:
        return state.generate_actions()
    
    def result(self, state: StateRepresentation, action: EnergyOperator) -> StateRepresentation:
        return state.apply_action(action)
    
    def value(self, state: StateRepresentation) -> float:
        return -state.heuristic()
        
    def goal_test(self, state: StateRepresentation) -> bool:
        return False

In [6]:
def generate_initial_state(params: ProblemParameters) -> StateRepresentation:    
    v_a=[]
    index_central = 0
    for client_i in params.clients:
        v_a.append(index_central)
    return StateRepresentation(params, v_a)

In [14]:
def generate_initial_state2(params: ProblemParameters) -> StateRepresentation:    
    v_a=[]    
    for client_i in range(params.num_clients):    
        index_central= params.matriu_c[client_i].index(min(params.matriu_c[client_i]))
        v_a.append(index_central)
        
    return StateRepresentation(params, v_a)

In [8]:
def generate_initial_state3(params: ProblemParameters) -> StateRepresentation:    
    v_a=[]    
    for client_i in range(params.num_clients):    
        index_central= params.matriu_c[client_i].index(max(params.matriu_c[client_i]))
        v_a.append(index_central)
        
    return StateRepresentation(params, v_a)

In [19]:
def generate_initial_state4(params: ProblemParameters) -> StateRepresentation:    
    v_a=[] 
    consum=0
    central_ocupada=[]
    for client_i in range(params.num_clients):
        centrals= sorted(params.matriu_c[client_i])
        central_actual=0
            
        if params.centrals[central_actual].Produccion/2 > consum+ (params.matriu_c[client_i] ):
            index_central= centrals[central_acutal]
            
        else:
            while params.matriu_c[client_i].index(centrals[central_actual]) in central_ocupada and central_actual<len(params.matriu_c[client_i]):
                central_acutal+=1
                
            index_central= params.matriu_c[client_i].index(centrals[central_actual]) 
        
        v_a.append(index_central)
        
    return StateRepresentation(params, v_a)

In [21]:
def generate_initial_state5(params: ProblemParameters) -> StateRepresentation:    
    v_a=[] 
    consum=0
    central_ocupada=[]
    for client_i in range(params.num_clients):
        centrals= sorted(params.matriu_c[client_i])
        central_actual=0
            
        if params.centrals[central_actual].Produccion/2 > consum+ (params.matriu_c[client_i] ):
            index_central= centrals[ccentral_actual]
            
        else:
            while params.matriu_c[client_i].index(centrals[central_actual]) in central_ocupada and central_actual<len(params.matriu_c[client_i]):
                central_acutal+=1
                
            index_central= params.matriu_c[client_i].index(centrals[central_actual]) 
        
        v_a.append(index_central)
        
    return StateRepresentation(params, v_a)

In [22]:
initial_state = generate_initial_state4(ProblemParameters(20, 500,clientes,centrales))

UnboundLocalError: local variable 'central_acutal' referenced before assignment

In [17]:
initial_state

v_a=[7, 5, 2, 1, 8, 8, 1, 3, 5, 5, 8, 6, 13, 10, 13, 3, 5, 1, 10, 2, 0, 10, 17, 1, 8, 6, 1, 5, 8, 10, 3, 4, 3, 4, 14, 0, 17, 6, 5, 2, 0, 2, 15, 7, 8, 0, 11, 8, 1, 6, 11, 1, 7, 2, 10, 0, 0, 2, 4, 8, 13, 1, 15, 3, 2, 0, 2, 2, 12, 8, 12, 17, 8, 19, 16, 5, 5, 18, 8, 8, 10, 12, 1, 18, 5, 18, 1, 5, 10, 2, 17, 18, 1, 8, 3, 1, 7, 13, 5, 5, 1, 11, 5, 7, 11, 10, 7, 18, 8, 12, 4, 18, 4, 19, 12, 3, 2, 7, 1, 8, 1, 6, 6, 8, 5, 13, 2, 1, 17, 12, 18, 5, 8, 0, 5, 5, 4, 7, 4, 17, 6, 4, 8, 3, 8, 7, 16, 1, 3, 2, 4, 8, 10, 12, 1, 8, 4, 14, 12, 5, 8, 5, 12, 8, 1, 3, 4, 18, 8, 0, 0, 7, 10, 3, 0, 5, 13, 7, 3, 12, 4, 1, 3, 17, 1, 10, 1, 1, 3, 1, 2, 7, 19, 8, 3, 3, 4, 0, 6, 11, 5, 17, 12, 8, 7, 13, 1, 10, 13, 2, 7, 5, 11, 13, 8, 7, 3, 7, 0, 16, 10, 0, 13, 5, 1, 6, 0, 1, 5, 10, 5, 2, 4, 10, 1, 2, 0, 1, 8, 13, 18, 11, 1, 18, 7, 10, 5, 12, 8, 18, 18, 7, 8, 8, 3, 2, 8, 4, 5, 2, 3, 8, 5, 5, 7, 6, 4, 2, 7, 3, 10, 5, 18, 2, 0, 5, 6, 12, 8, 5, 18, 7, 3, 1, 6, 13, 3, 7, 7, 8, 7, 8, 15, 7, 12, 7, 1, 3, 18, 12, 8, 10, 1, 

In [18]:
n_count = hill_climbing(EnergyProblem(initial_state))
t_count = timeit.timeit(lambda: hill_climbing(EnergyProblem(initial_state)), number=5)

print(f"heuristic_count,   benefici : {n_count.v_a} contenidors")
print(f"heuristic_count,   temps/5 execs.  : {t_count}s.")

KeyboardInterrupt: 

In [ ]:
benefici=0
for index_client, index_central in enumerate(n_count.v_a):
            
    client=n_count.params.clients[index_client]
    central=n_count.params.centrals[index_central]
            
    consumMonetitzable = client.Consumo
    consumTotalClient = n_count.params.matriu_c[index_client][index_central]
            
    garant_nogarant = VEnergia.tarifa_cliente_no_garantizada if client.Contrato else VEnergia.tarifa_cliente_garantizada
    costesProd= VEnergia.costs_production_mw(central.Tipo)

            
    #tarifa_cliente_penalizacion
            
    benefici+= consumMonetitzable * garant_nogarant(client.Tipo) - (consumTotalClient*costesProd)
    
for index_central in set(n_count.v_a):
    central=n_count.params.centrals[index_central]
    benefici-= VEnergia.daily_cost(central.Tipo)

print(benefici)
    
    


In [ ]:
benefici=0
l=[4, 2, 4, 3, 4, 4, 3, 3, 2, 2, 4, 4, 3, 4, 4, 4, 2, 4, 4, 4, 3, 4, 4, 4, 2, 3, 4, 2, 4, 2, 4, 4, 3, 3, 4, 3, 2, 4, 2, 4, 3, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 4, 2, 2, 4, 3, 3, 2, 4, 4, 4, 4, 2, 4, 4, 3, 2, 2, 4, 2, 2, 4, 2, 4, 4, 2, 3, 3, 4, 4, 4, 4, 3, 3, 2, 4, 3, 2, 2, 2, 4, 4, 4, 4, 4, 3, 2, 3, 4, 4, 4, 3, 4, 4, 4, 2, 3, 3, 2, 4, 3, 2, 3, 4, 4, 3, 4, 4, 3, 2, 4, 3, 2, 4, 4]
for index_client, index_central in enumerate(l):
            
    client=n_count.params.clients[index_client]
    central=n_count.params.centrals[index_central]
            
    consumMonetitzable = client.Consumo
    consumTotalClient = n_count.params.matriu_c[index_client][index_central]
            
    garant_nogarant = VEnergia.tarifa_cliente_no_garantizada if client.Contrato else VEnergia.tarifa_cliente_garantizada
    costesProd= VEnergia.costs_production_mw(central.Tipo)

            
    #tarifa_cliente_penalizacion
            
    benefici+= consumMonetitzable * garant_nogarant(client.Tipo) - (consumTotalClient*costesProd)
    
for index_central in set(n_count.v_a):
    central=n_count.params.centrals[index_central]
    benefici-= VEnergia.daily_cost(central.Tipo)

print(benefici)